In [2]:
import scanpy as sc
import anndata as an
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import torch
from sys import getsizeof
import json 
import scanpy as sc 

from torch.utils.data import DataLoader

import os, sys
sys.path.append('../src')

from data import *
from model import *
# from testing import *
from lightning_train import *

In [ ]:
datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad']
labelfiles=['../data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv']
le = LabelEncoder()

mo_data = an.read_h5ad('../data/mouse/Mo_PV_paper_TDTomato_mouseonly.h5ad')
atlas_data = an.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [ ]:
reduced = sc.pl.umap(mo_data)

## Subcluster Model labels 

We need to classify the cells into just PVALB/SST, so we'll generate that here and train a local model below

In [3]:
second_model_labels = ['Ctx_LHX6/PVALB', 'Ctx_LHX6/SST']

sm_label_df = atlas_data.obs.reset_index(drop=True)

sm_label_df = sm_label_df[sm_label_df.reset_index(drop=True)['class'].isin(second_model_labels)]

sm_label_df.index.name = 'cell'
sm_label_df = sm_label_df.reset_index()
sm_label_df.to_csv('../data/mouse/pvalb_sst_labels.csv', index=False)

In [54]:
td_indices = np.where(mo_data.X.todense()[:, -1] > 0)[0]
to_pred = mo_data[td_indices]

In [55]:
td_indices

array([   9,   14,   18,   21,   33,   52,   54,   69,   71,   82,   83,
         85,   90,   98,  100,  122,  127,  128,  132,  134,  146,  147,
        158,  161,  163,  167,  173,  176,  191,  215,  220,  223,  224,
        230,  247,  250,  253,  264,  268,  269,  272,  273,  278,  279,
        293,  296,  298,  299,  301,  306,  309,  312,  316,  335,  348,
        350,  357,  358,  359,  364,  365,  366,  375,  376,  380,  381,
        394,  396,  400,  409,  410,  419,  434,  436,  444,  456,  457,
        458,  460,  461,  463,  467,  468,  473,  474,  483,  484,  488,
        489,  495,  509,  510,  514,  535,  540,  542,  547,  549,  563,
        564,  571,  573,  574,  575,  590,  593,  594,  595,  615,  618,
        626,  636,  653,  665,  667,  685,  688,  689,  705,  710,  713,
        715,  720,  727,  729,  734,  737,  740,  746,  747,  756,  757,
        762,  770,  771,  774,  794,  804,  810,  819,  828,  829,  834,
        835,  849,  850,  866,  868,  873,  876,  8

In [6]:
mo_genes = mo_data.var.index.values
atlas_genes = atlas_data.var.index.values

mo_genes = [x.strip().upper() for x in mo_genes]
atlas_genes = [x.strip().upper() for x in atlas_genes]

refgenes = sorted(list(set(mo_genes).intersection(atlas_genes)))
len(refgenes)

25163

In [7]:
trainlabels = pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(trainlabels['class'])

In [9]:
results = []

dataset = TestAnndatasetMatrix(mo_data.X)
loader = DataLoader(dataset, batch_size=4, num_workers=0)

In [10]:
model = SIMSClassifier.load_from_checkpoint(
    '../checkpoints/checkpoint-40-desc-mostajo-mouse-5-23-22.ckpt',
    input_dim=len(refgenes),
    output_dim=50,
)

Initializing network
Initializing explain matrix


In [41]:
from tqdm import tqdm
import torch.nn.functional as f

topk_td = []
topk_all = []

to_pred = mo_data[np.where(mo_data.X.todense()[:, -1] > 0)[0]]
to_pred = TestAnndatasetMatrix(to_pred.X)

all_data = TestAnndatasetMatrix(mo_data.X)

all_loader = DataLoader(all_data, batch_size=4, num_workers=0)
loader = DataLoader(to_pred, batch_size=4, num_workers=0)

model.eval()
with torch.no_grad():
    for X in tqdm(loader):
        X = clean_sample(
            sample=X,
            refgenes=refgenes,
            currgenes=mo_genes,
        )

        res, _ = model(X)
        
        _, top_preds = res.topk(3, axis=1) # to get indices
        topk_td.extend(top_preds.numpy())
        
    for X in tqdm(all_loader):
        X = clean_sample(
            sample=X,
            refgenes=refgenes,
            currgenes=mo_genes,
        )

        res, _ = model(X)
        
        _, top_preds = res.topk(3, axis=1) # to get indices
        topk_all.extend(top_preds.numpy())

1
1


In [42]:
topk_all = pd.DataFrame(topk_all)
topk_td = pd.DataFrame(topk_td)

In [43]:
topk_all

,0,1,2
0,0,4,13
1,0,4,15
2,33,26,16
3,28,23,21
4,36,13,4
...,...,...,...
1619,27,28,26
1620,28,23,21
1621,44,45,42
1622,4,7,48


In [59]:
import plotly.express as px 
import plotly.graph_objects as go

first = pd.Series(le.inverse_transform(topk_td.loc[:, 0].values)).value_counts()
second = pd.Series(le.inverse_transform(topk_td.loc[:, 1].values)).value_counts()

fig = go.Figure(
    data=[
        go.Bar(x=first.index, y=first.values, name='First Predicted Label'),
#         go.Bar(x=second.index, y=second.values, name='Second Predicted Label')
    ]
)

fig.update_layout(
    title='Distribution of Inferred Cell Types from Transplanted Cells (TDTomato Positive)',
    font_family='Serif',
    xaxis_title='Cell Type',
    yaxis_title='Number of cells',
    legend=None
)

fig.show()
fig.write_image('../../ms-thesis/images/mo_experiment_inference_td.pdf', scale=3)

In [61]:
first = pd.Series(le.inverse_transform(topk_all.loc[:, 0].values)).value_counts()
second = pd.Series(le.inverse_transform(topk_all.loc[:, 1].values)).value_counts()

fig = go.Figure(
    data=[
        go.Bar(x=first.index, y=first.values, name='First Predicted Label'),
#         go.Bar(x=second.index, y=second.values, name='Second Predicted Label')
    ]
)

fig.update_layout(
    title='Distribution of Inferred Cell Types from Transplanted Cells (TDTomato Positive)',
    font_family='Serif',
    xaxis_title='Cell Type',
    yaxis_title='Number of cells',
    legend=None,
)

fig.show()
fig.write_image('../../ms-thesis/images/mo_experiment_inference_all.pdf', scale=3)

Let's also visualize the training distribution to understand the differnces in cell type.

In [23]:
MGE_cluster = np.where(labels == 'MGE_LHX6/MAF')[0]

MGE_cluster

array([ 122,  280,  343,  344,  361,  365,  384,  413,  443,  454,  489,
        520,  604,  710,  772,  797,  874, 1032, 1047, 1082, 1134, 1158,
       1189, 1221, 1247, 1272, 1334, 1410, 1454, 1474, 1527, 1585, 1603,
       1619])

In [25]:
second_confident = pd.Series(le.inverse_transform(topk.loc[MGE_cluster, 1]))

px.bar(second_confident.value_counts())

In [26]:
second_confident = pd.Series(le.inverse_transform(topk.loc[MGE_cluster, 2]))

px.bar(second_confident.value_counts())

# Model training with just PVALB and SST

In [ ]:
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl 

module = DataModule(
    datafiles=['../data/mouse/MouseAdultInhibitoryNeurons.h5ad'],
    labelfiles=['../data/mouse/pvalb_sst_labels.csv'],
    class_label='class',
    batch_size=32,
    num_workers=0,
    shuffle=True,
    drop_last=True,
    normalize=True,
    refgenes=refgenes,
    currgenes=atlas_genes,
    deterministic=True,
)

wandb_logger = WandbLogger(
    project=f"Mostajo Mouse Model, PVALB vs SST",
    name='Local',
)

lr_callback = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

upload_callback = UploadCallback(
    path='../checkpoints',
    desc='mostajo-mouse-5-23-22-pvalb-sst'
)

early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
)

trainer = pl.Trainer(
    gpus=(1 if torch.cuda.is_available() else 0),
    auto_lr_find=False,
    logger=wandb_logger,
    max_epochs=500,
    gradient_clip_val=0.5,
    callbacks=[
        lr_callback, 
        upload_callback,
        early_stopping_callback,
    ]
)

model = SIMSClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
)

trainer.fit(model, datamodule=module)

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning:

There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Initializing network
Initializing explain matrix
Creating train/val/test DataLoaders...
